In [3]:
import pandas as pd
import json
import os
import numpy as np
from folders_tb import read_json_to_dict
from mysql_driver import MySQL
import mysql.connector
from sqlalchemy import create_engine

In [4]:
SEP = os.sep
dir = os.path.dirname
csv_fullpath = dir(dir(os.getcwd())) + SEP + "data" + SEP + "SP120.csv" #put name of file here

In [5]:
json_readed = read_json_to_dict("sql_server_settings.json")
IP_DNS = json_readed["IP_DNS"]
USER = json_readed["USER"]
PASSWORD = json_readed["PASSWORD"]
BD_NAME = json_readed["BD_NAME"]
PORT = json_readed["PORT"]

In [6]:
mysql_db = MySQL(IP_DNS=IP_DNS, USER=USER, PASSWORD=PASSWORD, BD_NAME=BD_NAME, PORT=PORT)
conn = mysql_db.connect()

Connected to MySQL server [x9808164k_ds_april_2021_db]


In [1]:

def footballcleanandsendtomysql(COUNTRY, LEAGUE, SEASON, csvpath):
    df = pd.read_csv(csvpath, index_col=None)
    df = df[['Date','HomeTeam','FTHG','AwayTeam','FTAG','B365H','B365D','B365A']]
    df['Country'] = COUNTRY
    df['League'] = LEAGUE
    df['Season'] = SEASON
    df['Homeoraway'] = "Home"
    df.rename(columns={'HomeTeam':'Team', 'FTHG':'TeamGoals', 'AwayTeam':'Opponent', 'FTAG':'OpponentGoals', 'B365H':'TeamOdds', 'B365D':'DrawOdds',        'B365A':'OpponentOdds'}, inplace=True)
    awaydf = df.rename(columns={'Team':'Opponentt', 'TeamGoals':'OpponentGoalst', 'Opponent':'Teamt', 'OpponentGoals':'TeamGoalst', 'TeamOdds':'OpponentOddst', 'OpponentOdds':'TeamOddst'})
    awayt = awaydf.rename(columns={'Opponentt':'Opponent', 'OpponentGoalst':'OpponentGoals', 'Teamt':'Team', 'TeamGoalst':'TeamGoals', 'OpponentOddst':'OpponentOdds', 'TeamOddst':'TeamOdds'})
    awayt["Homeoraway"].replace({"Home": "Away"}, inplace=True)
    awayt = awayt[["Date", "Team", "TeamGoals", "Opponent", "OpponentGoals", "TeamOdds", "DrawOdds", "OpponentOdds", "Country", "League", "Season", "Homeoraway"]]
    df = df.append(awayt)
    conditions = [(df['TeamGoals'] > df['OpponentGoals']), 
    (df['TeamGoals'] == df['OpponentGoals']),
    (df['TeamGoals'] < df['OpponentGoals'])]
    values = ['Win', 'Draw', 'Lose']
    df['Result'] = np.select(conditions, values)
    conditions = [(df['Result'] == "Win"),
    (df['Result'] != "Win")]
    values = [(df['TeamOdds']-1), -1]
    df['ResultBetWin'] = np.select(conditions, values)
    conditions = [(df['Result'] == "Draw"),
    (df['Result'] != "Draw")]
    values = [(df['DrawOdds']-1), -1]
    df['ResultBetDraw'] = np.select(conditions, values)
    conditions = [(df['Result'] == "Lose"),
    (df['Result'] != "Lose")]
    values = [(df['OpponentOdds']-1), -1]
    df['ResultBetLose'] = np.select(conditions, values)
    conditions = [(df['TeamOdds'] <= df['OpponentOdds']),
    (df['TeamOdds'] > df['OpponentOdds'])]
    values = ["Favourite", "NotFavourite"]
    df['Favourite'] = np.select(conditions, values)
    df['TotalGoals'] = df['TeamGoals'] + df['OpponentGoals']
    df = df[["Country", "League", "Season", "Date", "Homeoraway", "Team", "TeamGoals", "Opponent", "OpponentGoals", "Favourite", "Result", "TotalGoals", "TeamOdds", "DrawOdds", "OpponentOdds", "ResultBetWin", "ResultBetDraw", "ResultBetLose"]]
    df['Date'] = pd.to_datetime(df['Date'])
    engine = create_engine(f'mysql+pymysql://{USER}:{PASSWORD}@{IP_DNS}:{PORT}/{BD_NAME}')
    df.to_sql('Futbol', con = engine, index=False, if_exists='append')
    print(csvpath, "Cleaned and uploaded to mysql")
    return df


In [ ]:
# footballcleanandsendtomysql('SPAIN', 'LALIGA', '20/21', csv_fullpath) example of function call

In [15]:
#cursor = conn.cursor()
#cursor.execute("DROP TABLE IF EXISTS Futbol")
#conn.close()